# Elevated-plus maze analysis

In [ ]:
%load_ext rpy2.ipython
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype
import seaborn as sns
import scipy.stats
from scipy.ndimage import filters
import os
import time
import itertools

import custom

In [ ]:
idx = pd.IndexSlice

## Import and setup data

In [ ]:
# Parameters
filename = './data/epm.h5'

In [ ]:
# Import data
with pd.HDFStore(filename, mode='r') as hf:
    neural_activity = hf['neural']
    behav = hf['behav']

# If running Python 3
import sys
if sys.version_info[0] <= 3:
    neural_activity.columns.names = [x.decode() for x in neural_activity.columns.names]
    behav.columns.names = [x.decode() for x in behav.columns.names]

Normalize neural data by dividing standard deviation within each neuron.

In [ ]:
# Normalize imaging data

neural_norm = neural_activity * (1 / neural_activity.std())

# Remove cells with no activity/std in baseline (if any)
neural_norm = neural_norm.replace([np.inf, -np.inf], np.nan)
neural_norm = neural_norm.dropna(axis=1, how='all')

n_cells = neural_norm.shape[1]

In [ ]:
# Normalize features

col_no_norm = [
    'In zone(Center / center-point)',
    'In zone(Closed arms / center-point)',
    'In zone(Open arms / center-point)',
    'Mobility state(Immobile)',
    'Mobility state(Mobile)',
    'Mobility state(Highly mobile)',
    'Zone transition(center-point / Center > Closed arms)',
    'Zone transition(center-point / Center > Open arms)',
    'Zone transition(center-point / Closed arms > Center)',
    'Zone transition(center-point / Open arms > Center)',
]

behav_norm = behav.copy()
for column in behav.columns:
    if column[1] not in col_no_norm:
        behav_norm[column] = (behav[column] - behav[column].mean()) * (1 /  (behav[column].std() or 1))
        # if max is 0, entire column is 0

### Bin neural activity

In [ ]:
n_bins = 10
max_d = 35
bins = np.arange(n_bins + 1, dtype=float) / n_bins * max_d
bins = np.insert(bins, 0, -1)
bin_labels = np.arange(0, n_bins+1) / 10.
binned_loc = pd.cut(behav.xs('Distance to zone', axis=1, level='feature').stack('subject'), bins, labels=bin_labels).unstack('subject')
binned_loc.columns = pd.MultiIndex.from_product([binned_loc.columns, ['Distance to zone']], names=['subject', 'feature'])

df_zone = behav.loc[:, (slice(None), ['In zone(Center / center-point)', 'In zone(Closed arms / center-point)', 'In zone(Open arms / center-point)'])].astype(bool).sort_index(axis=1)
for subj, df in df_zone.groupby(axis=1, level='subject'):
    df_zone[(subj, 'zone')] = pd.Series(['center' if x else 'closed' if y else 'open' if z else 'na' for (x, y, z), in zip(df.values)], index=df.index)

df_loc = pd.concat([binned_loc, df_zone], axis=1).sort_index(axis=1)

loc = df_loc.loc[:, (slice(None), ['zone', 'Distance to zone'])]
dfs = {}
for subj, df in df_loc.groupby(axis=1, level='subject'):
    dfs[subj] = pd.concat([loc[subj], neural_norm[subj]], axis=1).groupby(['Distance to zone', 'zone']).mean()

neural_zone = pd.concat(dfs, axis=1, names=['subject', 'neuron'])

### Group neurons

In [ ]:
# Average activity by location

n_cells = neural_norm.shape[1]
p_thresh = 0.05 / n_cells #neural_arm.shape[0]
center_col = 'In zone(Center / center-point)'
closed_col = 'In zone(Closed arms / center-point)'
open_col = 'In zone(Open arms / center-point)'

# Average activity for each neuron in each zone
neural_arm = pd.DataFrame(
    index=neural_norm.columns,
    columns=['center', 'open', 'closed'],
    dtype=float
)
neural_arm.columns.name = 'arm'
neural_response = pd.DataFrame(index=['group', 'difference', 'p'], columns=neural_norm.columns, dtype=float)

for subj, df in neural_norm.groupby(axis=1, level='subject'):
    center_ix = behav_norm[(subj, center_col)] == 1
    closed_ix = behav_norm[(subj, closed_col)] == 1
    open_ix = behav_norm[(subj, open_col)] == 1
    
    neural_arm.loc[subj, 'center'] = df.loc[center_ix, :].mean(axis=0)
    neural_arm.loc[subj, 'closed'] = df.loc[closed_ix, :].mean(axis=0)
    neural_arm.loc[subj, 'open'] = df.loc[open_ix, :].mean(axis=0)
    
    for cell in df.loc[:, subj]:
        X1 = df.loc[closed_ix, (slice(None), cell)]
        X2 = df.loc[open_ix, (slice(None), cell)]
        _, neural_response.loc['p', (subj, cell)] = scipy.stats.mannwhitneyu(X1, X2)

# Calculate difference between arms
neural_response.loc['difference'] = (neural_arm['open'] - neural_arm['closed'])
significant_ix = neural_response.loc['p'] < p_thresh

# Define groups
neural_response.loc['group'] = 0
exc_ix = neural_response.loc['difference'] > 0
inh_ix = neural_response.loc['difference'] < 0
neural_response.loc['group', neural_response.columns[significant_ix & exc_ix]] = 1
neural_response.loc['group', neural_response.columns[significant_ix & inh_ix]] = -1

# Add groups to `neural_arm`
neural_arm.index = pd.MultiIndex.from_tuples(
    [x + (neural_response.loc['group', x], ) for x in neural_arm.index],
    names = ['subject', 'neuron', 'response']
)

In [ ]:
groups = neural_response.loc['group', :].astype(int).to_dict()
neural_norm_grp = neural_norm.copy()
neural_norm_grp.columns = pd.MultiIndex.from_tuples([x + (groups[x], ) for x in neural_norm], names=neural_norm.columns.names + ['group'])

## Correlations

Wilcoxon test is different between R and Python. R should be better.

In [ ]:
# Calculate distance-activity correlations

corr_method = scipy.stats.spearmanr

features = ['Distance to zone', 'Velocity']
neural_corr = pd.DataFrame(
    index=neural_norm_grp.columns,
    columns=pd.MultiIndex.from_product([['open', 'closed'], features, ['r', 'p']], names=['arm', 'feature', 'stat']),
    dtype=float
)
neural_corr_stats = pd.DataFrame(
    index=[-1, 0, 1],
    columns=pd.MultiIndex.from_product([['open', 'closed'], features, ['t', 'p']], names=['arm', 'feature', 'stat'])
)

for feature in features:
    for subj, df_subj in neural_norm_grp.groupby(axis=1, level='subject'):
        closed_ts = behav.index[behav[subj, 'In zone(Closed arms / center-point)'] == 1]
        open_ts = behav.index[behav[subj, 'In zone(Open arms / center-point)'] == 1]
        for cell in df_subj:
            for arm, arm_ix in zip(['open', 'closed'], [open_ts, closed_ts]):
                X = df_subj.loc[arm_ix, cell]
                Y = behav[subj, feature][arm_ix]
                ix = ~(X.isna() | Y.isna())

                neural_corr.loc[cell, (arm, feature, 'r')], neural_corr.loc[cell, (arm, feature, 'p')] = corr_method(X[ix], Y[ix])

# Stats on correlation distributions
for arm in ['closed', 'open']:
    for feature in features:
        for grp in [-1, 0, 1]:
            neural_corr_stats.loc[grp, (arm, feature)] = list(scipy.stats.wilcoxon(
                np.arctanh(neural_corr.loc[idx[:, :, grp], (arm, feature, 'r')]),
            ))

# Visualization

## Behavior

In [ ]:
cols = ['In zone(Closed arms / center-point)', 'In zone(Open arms / center-point)', 'In zone(Center / center-point)']
data = behav.loc[:, idx[:, cols]].mean()
data.name = 'time'
data = data.rename(index={
    'In zone(Center / center-point)': 'center',
    'In zone(Open arms / center-point)': 'open',
    'In zone(Closed arms / center-point)': 'closed'
})

data_ratio = data.unstack('feature')['open'] / data.unstack('feature').sum(axis=1)
t, p = scipy.stats.ttest_1samp(data_ratio, 0.5)

fig, axes = plt.subplots(ncols=2)
sns.stripplot(ax=axes[0], data=data.reset_index(), x='feature', y='time', order=['closed', 'center', 'open'], hue='subject')
sns.pointplot(ax=axes[0], data=data.reset_index(), x='feature', y='time', order=['closed', 'center', 'open'], palette=['k'], ci=68, join=False)
axes[0].get_legend().remove()

axes[1].pie([data_ratio.mean(), 1 - data_ratio.mean()], labels=['open', 'closed'], autopct='%1.1f%%',)
centre_circle = plt.Circle((0, 0), 1 / 1.4142, color='none', fc='white')
axes[1].add_artist(centre_circle)
axes[1].set_title('one-sample t test (0.5 mean)\nt: {}\np: {}'.format(t, p))
axes[1].axis('equal')

## Neural

In [ ]:
fig, axes = plt.subplots(ncols=2)

custom.cdf(neural_arm['closed'].values, ax=axes[0], label='closed')
custom.cdf(neural_arm['open'].values, ax=axes[0], label='open')
axes[0].legend()
axes[0].set_xlabel('mean_response')

data = neural_arm[['closed', 'open']]
ax0 = sns.stripplot(ax=axes[1], data=data.melt(), x='arm', y='value', s=3, jitter=True, zorder=0)
ax1 = sns.pointplot(ax=axes[1], data=data.melt(), x='arm', y='value', scale=0.5, color='k', ci=68, join=False)
# axes[1].errorbar(range(2), data.mean(), data.sem(), c='k', fmt='_', zorder=100)

test_val, p_val = scipy.stats.wilcoxon(neural_arm['open'], neural_arm['closed'])
fig.suptitle('Wilcoxon signed-rank test\nW: {}\np: {}'.format(test_val, p_val))
fig.tight_layout(rect=[0, 0.03, 1, 0.95])

In [ ]:
t, p = scipy.stats.ttest_rel(
    neural_arm['open'],
    neural_arm['closed']
)

data = neural_arm.stack().reset_index().rename(columns={0: 'neural'})
arm_type = CategoricalDtype(categories=['closed', 'center', 'open'], ordered=True)
data['arm'] = data['arm'].astype(arm_type)

fig, ax = plt.subplots()
sns.pointplot(
    data=data, x='arm', y='neural', hue='response',
    order=['closed', 'center', 'open'], hue_order=[-1, 0, 1],
    palette=['g', 'gray', 'b'], ci=68, ax=ax
)

In [ ]:
counts = neural_arm.groupby('response').count()['center']

fig, ax = plt.subplots()
counts.plot.pie(ax=ax, colors=['gray', np.array([60, 179, 113])/255., np.array([24, 116, 205])/255.], autopct='%1.1f%%', wedgeprops={'lw': 5, 'ec': 'w'})
ax.axis('image')
centre_circle = plt.Circle((0, 0), 1 / 1.4142, color='none', fc='white')
ax.add_artist(centre_circle);

In [ ]:
data = neural_zone.copy()
data = data.drop(itertools.product(bin_labels[1:], ['center']))
data = data.drop(itertools.product([0.0], ['closed', 'open']))
data = data.drop('na', level='zone')

n_rows = data.shape[0]
data = data.iloc[list(range(n_rows-2, 0, -2)) + list(range(0, n_rows, 2))]

sort_ix_peak = np.nanargmax(data.values, axis=0).argsort()
sort_ix = data.mean(axis=0, level='zone')
sort_ix = sort_ix.loc['open'] / sort_ix.loc['closed']
sort_ix_mean = sort_ix.values.argsort()

gridspec_kw = {'height_ratios': [5, 1], 'width_ratios': [1, 1, 0.1]}
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(10, 8), sharex='col', gridspec_kw=gridspec_kw)
sns.heatmap(data.iloc[:, sort_ix_peak].T, ax=axes[0, 0], center=0, vmax=2, cmap=cmap, cbar_ax=axes[0, 2]);
sns.heatmap(data.iloc[:, sort_ix_mean].T, ax=axes[0, 1], center=0, vmax=2, cmap=cmap, cbar=False);

data_long = data.copy()
data_long.columns = pd.MultiIndex.from_tuples(
    [x + (y, ) for x, y in zip(data.columns, neural_response.loc['group', data.columns].astype(int))],
    names=data.columns.names + ['group']
)
data_long = data_long.stack(list(range(data_long.columns.nlevels))).reset_index().rename(columns={0: 'neural'})
data_long['arm_loc'] = data_long['zone'] + ' ' + data_long['Distance to zone'].astype(str)
zone_order = (
    [' '.join(x) for x in itertools.product(['closed'], bin_labels[-1:0:-1].astype(str))] +
    ['center 0.0'] +
    [' '.join(x) for x in itertools.product(['open'], bin_labels[1:].astype(str))]
)
sns.lineplot(data=data_long, x='arm_loc', y='neural', hue='group',
             hue_order=[-1, 0, 1], palette=['g', 'gray', 'b'], ci=68, ax=axes[1, 0], sort=False)
sns.pointplot(data=data_long, x='arm_loc', y='neural', hue='group', join=False,
              order=zone_order, hue_order=[-1, 0, 1], palette=['g', 'gray', 'b'], ci=68, ax=axes[1, 1])
for ax in axes[1]:
    plt.setp(ax.get_xticklabels(), rotation=90)

fig.tight_layout()

## Correlations

In [ ]:
# Plot
neural_corr_long = neural_corr.xs('r', axis=1, level='stat').stack(list(range(neural_corr.columns.nlevels - 1))).reset_index().rename(columns={0: 'r'})
g = sns.FacetGrid(
    neural_corr_long, row='feature', col='arm', hue='group',
    hue_order=[-1, 0, 1], hue_kws={'color': ['g', 'gray', 'b']}, aspect=1.25
)
g.map(custom.cdf, 'r').add_legend()

# Format
for ax, (_, grp_df) in zip(g.axes.flatten(), neural_corr_stats.groupby(level=['feature', 'arm'], axis=1)):
    grp_df.columns = grp_df.columns.droplevel(['feature', 'arm'])
    ax.set_xlim(-0.75, 0.75)
    ax.set_title(
        f'{ax.get_title()}\n'
        f"inh | W: {grp_df.loc[-1, 't']:.2e} p: {grp_df.loc[-1, 'p']:.2e}\n"
        f"nc | W: {grp_df.loc[0, 't']:.2e} p: {grp_df.loc[0, 'p']:.2e}\n"
        f"exc | W: {grp_df.loc[1, 't']:.2e} p: {grp_df.loc[1, 'p']:.2e}\n"
    )
g.fig.tight_layout()